## 文本预处理
- 分词
- 去停用词
- 保存train.txt,test.txt

In [41]:
import sys
import os
import math
import random
import collections
import jieba

DATA_PATH = '../sub-THUCNews'
STOP_SIGNALS = {'\n','\u3000','\xa0',' ',}
def load_stop_words(stop_words_path='stopword.txt',stop_signals={'\n','\u3000','\xa0',' ',}):
    with open(stop_words_path,encoding='UTF-8-sig') as f:
        s_stop_words = set()
        for line in f:
            s_stop_words.add(line.strip())
        s_stop_words.update(stop_signals)
        return s_stop_words
    
def tokenization(text):
    '''
    给定一段文本，返回分词结果
    '''
    return jieba.cut(text)

def process(save_path, sub_path, s_stop_words,category, l_files):
    res_f = open(save_path,'a',encoding='UTF-8-sig')
    for file in l_files:
            with open(sub_path+'/'+file,'r',encoding='utf-8') as f:
                token_res = []          
                for word in tokenization(f.read()):
                    if word not in s_stop_words:
                        token_res.append(word)
                res_f.write(category+' '+ ' '.join(token_res)+'\n')
                
    res_f.close()
    
def text_preprocessing(data_path='../sub-THUCNews'):
    '''
    读取文件数据，进行分词，去停用，保存
    '''
    s_stop_words = load_stop_words()
    l_categories = os.listdir(data_path)
    for category in l_categories:
        sub_path = data_path + '/'+category
        l_files = random.shuffle(os.listdir(sub_path))  
        process('train.txt',sub_path, s_stop_words,category, l_files[:int(0.8*len(l_files))])  # 生成训练集数据
        process('test.txt',sub_path, s_stop_words,category, l_files[int(0.8*len(l_files)):])  # 生成测试集数据
    
text_preprocessing()

## 统计信息
- 统计每个类别文档数量
- 统计单词-类别文档数量
- 统计每个文档中词频信息

In [82]:
import collections

l_categories = ['财经','彩票','房产','股票','家居','教育','科技','社会','时尚','时政','体育','星座','游戏','娱乐']

def info_statis():
    d_words_count = {}
    d_doc_count = {}
    l_words_counters = []
    with open('train.txt',encoding='UTF-8-sig') as f:
        for line in f:
            label, content = line.strip().split(' ',1)
            words_counter = collections.Counter(content.split(' '))
            l_words_counters.append(words_counter)
            if label in d_doc_count:
                d_doc_count[label] += 1
            else:
                d_doc_count[label] = 1
    
            for word in words_counter.keys():
                if word in d_words_count and label in d_words_count[word].keys():
                    # 已存在相应单词
                    d_words_count[word][label] += 1  # 属于label类且对应item的文档数量

                elif word in d_words_count and label not in d_words_count[word].keys():
                    d_words_count[word][label] = 1
                
                else:
                    d_words_count[word] = {}
                    d_words_count[word][label] = 1
    
    return {
        'd_words_count':d_words_count,
        'd_doc_count':d_doc_count,
        'l_words_counters':l_words_counters
    }

res = info_statis()
print(res['d_words_count']['女足'])
print(res['d_doc_count']['体育'])
print(res['l_words_counters'][:1])

{'体育': 19}
120
[Counter({'女足': 14, '韦迪': 5, '男足': 5, '请': 3, '外籍': 3, '商瑞华': 3, '永川': 3, '体育局': 3, '会议': 3, '出线': 2, '下课': 2, '召开': 2, '研讨': 2, '12': 2, '省市': 2, '中国女足': 2, '困难': 2, '本报': 1, '专电': 1, '记者': 1, '孙永军': 1, '明天': 1, '会同': 1, '洪臣': 1, '薛立': 1, '两位': 1, '主管': 1, '副': 1, '主任': 1, '前往': 1, '大会': 1, '参加': 1, '此次': 1, '加快': 1, '步伐': 1, '观摩': 1, '24': 1, '日': 1, '热身赛': 1, '之机': 1, '足管': 1, '领导班子': 1, '齐聚': 1, '高': 1, '与会者': 1, '国脚': 1, '足协': 1, '负责人': 1, '本次': 1, '两个': 1, '议题': 1, '一是': 1, '举国体制': 1, '难题': 1, '全运会': 1, '金牌': 1, '刺激': 1, '鼓励': 1, '二是': 1, '现实': 1, '球队': 1, '内外部': 1, '步调': 1, '昨天': 1, '表示': 1, '后备力量': 1, '薄弱': 1, '关注度': 1, '低': 1, '匮乏': 1, '国家队': 1, '队员': 1, '透露': 1, '物色': 1, '想着': 1, '难度': 1, '由国': 1, '管部': 1, '按部就班': 1, '想': 1, '关键': 1, '距离': 1, '亚洲杯': 1, '开战': 1, '一个月': 1, '解压': 1, '大战': 1, '前': 1, '折损': 1, '大将': 1, '马晓旭': 1, '比赛': 1, '很大': 1, '中国足协': 1, '肯定': 1})]


## 特征提取
- 特征选择  
    依据互信息选择每个类别前500个特征词作为特征项
- 特征权重计算    
    依据TF-IDF计算每个特征的权重  
- 特征和权重矩阵

In [147]:
import math
import json

def get_MI(word, category, d_words_count, d_doc_count):
    N = sum([v for v in d_doc_count.values()])
    try:
        A = d_words_count[word][category]
    except KeyError:
        A = 0 
    C = d_doc_count[category] - A
    B = N - d_doc_count[category]- A
    D = N - C - B -A
    return A*N/((A+C)*(A+B))

def feature_selection(d_words_count,d_doc_count):
    '''
    依据互信息最大的选择500个特征,返回特征语料库
    '''
#     d_words_count = {'财经':{},'彩票':{},'房产':{},'股票':{},'家居':{},'教育':{},'科技':{},'社会':{},'时尚':{},'时政':{}
#                      ,'体育':{},'星座':{},'游戏':{},'娱乐':{}}  # {'财经': {'油价': 23, '高企将': 1,...
    
    l_corpus = []    
    # 计算互信息
    catedory_word_MIs = {'财经':[],'彩票':[],'房产':[],'股票':[],'家居':[],'教育':[],'科技':[],'社会':[],'时尚':[],'时政':[],
                      '体育':[],'星座':[],'游戏':[],'娱乐':[]}
    
    
    for word in d_words_count.keys():
        max_MI, closest_category = 0, None
        for category in d_doc_count.keys():
            MI = get_MI(word,category,d_words_count, d_doc_count)
            if MI > max_MI:
                max_MI = MI 
                closest_category = category
        catedory_word_MIs[closest_category].append((word,max_MI)) 
    
    # 对各个类别选取前500个作为特征词
    for category in d_doc_count.keys():
        l_corpus.extend([x[0] for x in sorted(catedory_word_MIs[category], key=lambda x:x[1], reverse=True)[:50]])
    
    return l_corpus
    
def weight_calculation(l_corpus,l_words_counters,d_words_count,d_doc_count):
    '''
    依据TF-IDF值计算每个特征在分类中的权重，在贝叶斯中没用到
    '''
    all_counter = collections.Counter()
    for counter in l_words_counters:
        all_counter += counter
    
    N = sum([v for v in d_doc_count.values()])  # 总的文档数
    word_tf_idf = {}
    for word in l_corpus:
        idf = math.log(N/sum([v for v in d_words_count[word].values()]))
        word_tf_idf[word] = all_counter.get(word)*idf
    
    return word_tf_idf
    
    
def feature_extraction():
    '''
    遍历train.txt，统计每个词在每个类别中出现次数，利用互信息选择特征，TF-IDF值计算特征权重
    '''
    info = info_statis()
    l_corpus = feature_selection(info['d_words_count'], info['d_doc_count'])
#     d_features = weight_calculation(l_corpus,info['l_words_counters'],info['d_words_count'],info['d_doc_count']) # 每个特征及其对应的权重
#     return d_features
    return l_corpus

def save_features(features):
    with open('features.json','w', encoding='utf-8') as f:
        f.write(json.dumps(features))
features = feature_extraction()
save_features(features)
print(features)

['比赛', '前', '球队', '球员', '之后', '国家队', '体育讯', '新', '队', '联赛', '男篮', '对手', '足球', '俱乐部', '最后', '足协', '队员', '之前', '曾', '主教练', '来说', '一场', '中国足协', '教练', '赛季', '球迷', '训练', '亚运会', '热身赛', '肯定', '一次', '原因', '主帅', '无法', '中国男篮', '决定', '主任', '只能', '备战', '刚刚', '世界杯', '冠军', '韦迪', '曾经', 'CBA', '女足', '体育', '重新', '提出', '阶段', '讯', '新浪', '12', '观众', '音乐', '电视剧', '歌曲', '这次', '导演', '拍摄', '演唱会', '歌手', '透露', '故事', '2010', '电影', '文', '角色', '举行', '节目', '专辑', '歌迷', '此次', '唱', '卫视', '表演', '成', '该剧', '经典', '演绎', '人物', '演唱', '采访', '笑', '一部', '剧中', '情感', '听', '饰演', '戏', '发行', '邀请', '剧', '播出', '这部', '讲述', '首次', '主持人', '组合', '亮相', '地板', '家居', '木地板', '装饰', '环保', '实', '舒适', '核心', '装修', '厨房', '实木', '材料', '红', '开启', '家装', '电器', '智能', 'www', '营销', '绿色', '温馨', '宅', '德意', '诞生', '安装', '线条', '客厅', '木材', '装', '酒吧', '式', '餐厅', '复古', '奢华', '简约', '混搭小窝', '铺装', '橡', '森林', '一块', '清洁', '大气', '盛大', '背景墙', '专卖店', '188G', '久盛', '美式', '原味', '质感', '10', '期', '14', '奖金', '本期', '注', '奖', '彩票', '双色球', '20', '开奖', '30', '号码', '号', '开出', '头奖',

## 训练分类器
- 贝叶斯分类器

In [151]:
C = 14  # 设定的类的数量

import collections

def load_features():
    with open('features.json',encoding='utf-8') as f:
        features = json.loads(f.read())
    return features

def fit_transform(features, words_counter):
    '''
    将输入的文档依据特征集合转换为向量形式
    '''
    doc_feature_vector = []
    for feature in features:
        if words_counter.get(feature):
            doc_feature_vector.append(words_counter.get(feature))
        else:
            doc_feature_vector.append(0)
    return doc_feature_vector

def transform(features, file_path):
    '''
    接受特征字典{'feature':weight}和文件（训练或测试），将数据转换为向量表示返回
    '''
    d_categories = {}  # 统计每个类的数量信息
    l_category_vectors = []  # 统计每个类和特征向量信息
    with open(file_path, encoding='UTF-8-sig') as f:
        for line in f:
            label, content = line.strip().split(' ',1)
            words_counter = collections.Counter(content.split(' '))
            doc_feature_vector = fit_transform(d_features, words_counter)
            d_category_vector = {
                'category':label,
                'vector':doc_feature_vector
            }
            l_category_vectors.append(d_category_vector)
            try:
                d_categories[label] += 1
            except KeyError:
                d_categories[label] = 1
            
        return {
            'd_categories':d_categories,  # 每个类别的统计信息，用于计算先验概率
            'l_category_vectors':l_category_vectors  # 由类别和特征向量组成的字典列表
        }
           
features = load_features()
trans_res = transform(features,'train.txt')
print(trans_res['d_categories'])

{'体育': 120, '娱乐': 120, '家居': 120, '彩票': 120, '房产': 120, '教育': 120, '时尚': 120, '时政': 120, '星座': 120, '游戏': 120, '社会': 120, '科技': 120, '股票': 120, '财经': 120}


In [200]:
import jieba
import pickle
import json
from collections import Counter

def load_stop_words(stop_words_path='stopword.txt',stop_signals={'\n','\u3000','\xa0',' ',}):
    with open(stop_words_path,encoding='UTF-8-sig') as f:
        s_stop_words = set()
        for line in f:
            s_stop_words.add(line.strip())
        s_stop_words.update(stop_signals)
        return s_stop_words
    
def get_prior_prb(d_categories):
    '''
    计算先验概率，接受由所有分类以及对应数量组成的字典
    '''
    d_prior_prb = {}  # P(cj)
    N_all = sum(d_categories.values())
    for category,N_c in d_categories.items():
        d_prior_prb[category] = (1+N_c)/(C+N_all)
    
    return d_prior_prb

def get_likelihood_prb(l_category_vectors):
    '''
    计算似然概率，接受由category和特征向量组成的列表
    '''
    d_likelihood_prb = {}   # P(wi|cj)
    d_feature_category = {}  # 统计某一特征对应分类的分量
    d_all_category = {}  # 统计所有特征对应分类的分量
    for d_category_vector in l_category_vectors:
        category = d_category_vector['category']
        vector = d_category_vector['vector']
        for i,v in enumerate(vector):
            try:
                d_feature_category[(i,category)] += v
            except KeyError:
                d_feature_category[(i,category)] = v

            finally:
                if category not in d_all_category:
                    d_all_category[category] = v
                else:
                    d_all_category[category] += v

    M = len(vector)

    for k,v in d_feature_category.items():
        d_likelihood_prb[k] = (1+v)/(M + d_all_category[k[1]]) 
    return d_likelihood_prb

def get_predict_prb(prior_prb,likelihood_prb, categories, l_category_vectors):
    '''
    基于先验概率和条件概率计算
    '''
    count = 0 
    total = len(l_category_vectors)
    print('测试数据：{}条'.format(total))
    for d_category_vector in l_category_vectors:
        # 对每一条特征做分类
        true_category = d_category_vector['category']
        vector = d_category_vector['vector']
        i =0
        max_predict_prb = 0
        predict_category = None
        for category in categories:
            # 计算该文本在各个分类下的概率
            prod_likelihood_prb = 1
            for i,v in enumerate(vector):
                # 对向量中每一个不为0的分量求概率
                if v != 0 :
                    prod_likelihood_prb *= likelihood_prb[(i,category)]
            predict_prb = prior_prb[category]*prod_likelihood_prb
#             print('category:{},prb:{}'.format(category, predict_prb))
            if predict_prb > max_predict_prb:
                predict_category = category  # 更新概率最大类别
                max_predict_prb = predict_prb
#         print('true_category:{},predict_category:{}'.format(true_category,predict_category))
        if predict_category == true_category:
            count += 1
    print('准确率：{}'.format(count/total))
    
class NaiveBayes: 
    
    def __init__(self):
        self._features = self.__load_features()
        self._prior_prb = {}  # 各个分类的先验概率
        self._likelihood_prb = {}  # 各个特征在不同分类下的概率
        self._l_categories = ['财经','彩票','房产','股票','家居','教育','科技','社会','时尚','时政','体育','星座','游戏','娱乐']   
        
        self._train_path = 'train.txt'
        self._test_path = 'test.txt'
        
        self._s_stop_words = None
    
    def __load_features(self):
        with open('features.json',encoding='utf-8') as f:
            features = json.loads(f.read())
        return features
    
    def _tokenization(self,text):
        return jieba.cut(text)
    
    def _fit_transform(self,words_counter):
        '''
        将输入的文档依据特征集合转换为向量形式
        '''
        doc_feature_vector = []
        for feature in self._features:
            if words_counter.get(feature):
                doc_feature_vector.append(words_counter.get(feature))
            else:
                doc_feature_vector.append(0)
        return doc_feature_vector
    
    def _text_preprocess(self,text):
        '''
        预处理数据，返回文本的特征向量表达形式
        '''
        if not self._s_stop_words:
            self._s_stop_words = load_stop_words()
        token_res = []          
        for word in self._tokenization(text): # 分词
            if word not in self._s_stop_words:  # 去停用词
                token_res.append(word)
        words_counter = Counter(token_res)
        return self._fit_transform(words_counter)
        
    def train(self):
        print('training..')
        transform_res = transform(self._features,self._train_path)
        self._prior_prb = get_prior_prb(transform_res['d_categories'])
        self._likelihood_prb = get_likelihood_prb(transform_res['l_category_vectors'])
        
    def test(self):
        print('testing...')
        transform_res = transform(self._features, self._test_path)
        get_predict_prb(self._prior_prb, self._likelihood_prb, self._l_categories, transform_res['l_category_vectors'])
        
    def predict(self,text):
        vector = self._text_preprocess(text)
        max_predict_prb = 0
        for category in self._l_categories:
            # 计算该文本在各个分类下的概率
            prod_likelihood_prb = 1
            for i,v in enumerate(vector):
                # 对向量中每一个不为0的分量求概率
                if v != 0 :
                    prod_likelihood_prb *= self._likelihood_prb[(i,category)]
            predict_prb = self._prior_prb[category]*prod_likelihood_prb
#             print('category:{},prb:{}'.format(category, predict_prb))
            if predict_prb > max_predict_prb:
                predict_category = category  # 更新概率最大类别
                max_predict_prb = predict_prb
        print(predict_category)
        
        
naive_bayes = NaiveBayes()
naive_bayes.train()
naive_bayes.test()

TEST_TEXT ="""
婚恋心理：婆媳聊天必知的潜规则(图)
　　婆媳矛盾是我们中华民族的千古矛盾，一直都得不到缓解。这个社会的人都有两面性，大家嘴上说着一套一套的漂亮话，但是实际上所作所为，又是另外一回事。而婆媳关系也有两套规则，一套是明规则，还有一套潜规则，利用好了，这个千古矛盾对你来说将不再是难题。
　　婆媳相处如何妙用“潜规则”
　　可是，我们当中有多少人是口含银匙而生呢？多少人是公主下嫁招驸马的童话呢我们当中的大多数，不都是要为柴米油盐生计而喜怒哀乐吗？不都要正视如何和婆家人相处——我们不想可又不得不去做吗？
　　首先，我建议婆媳之间不要直接交流，有什么相左的意见应该通过先生缓冲一下，他是“汉奸”——会和皇军交流，也懂八路的心思。
　　其次，如果直接交流受阻，一定要先自省：自己冲不冲动，有没有言语不当的地方，对婆婆有没有肢体冲撞，自己如果和妈妈这么说，妈妈怎么反应如果这些都自省过了，没有问题，那就要和先生说，实事求是，注意方式，不要动怒说粗，宁可哭，不可以骂人。如果是自己做的过分，有形式上的不当之处，但是内容没有错，就得避重就轻一点了，但是要提。记住：提前招认，绝对好过后来被盘问不得不招。如果你有重大错误，对不起，我也不知道怎么办了。因为我从来不和婆婆正面交流不同意见。请其她姐妹指点吧。
　　总之，要做和先生解释说明冲突的第一人，要尽量心平气和，决不能搞人身攻击，婆婆丰满说人家是吹了气的青蛙，公公苗条说人家是榨干的甘蔗。要会做人，尤其是有外人在的场合，要表现的温和有礼，听话勤快，既让婆婆有面子，也可以请外人给你制造舆论。
　　婆媳相处，要善于利用“潜规则”
　　婆媳交流，要注意不能乱用潜规则，尽量说漂亮的官话。哪怕虚伪点，也不能来个赤裸裸的大实话，起码，不能首先使用大实话。聪明的妈妈会教女儿嘴巴要甜，说白了就是要会说官话。
　　当然，官话不仅仅是说话，还包括行动。例如一个五十多岁的媳妇得到了众人的赞扬，说她有媳妇相，自己都是有媳妇的人了，还那么孝顺婆婆。那她是怎么做的呢有客人来了，她贴身伺候婆婆，给婆婆拿着热水袋，香烟火柴，站在婆婆身边伺候着。其实，她这是在监视婆婆，让她没法说坏话，要说，只能说好话——这样，她的好名声就得到最权威的认可了。
　　如果娘家和婆家势力悬殊，或是先生靠着爸爸提携，你就不用担心什么婆媳关系了，婆婆哪还敢说你坏话她得为儿子好啊。这种情况下，媳妇若是为长久计，就要锦上添花，待公婆好一些，省得老公翅膀硬了老爹退休了，公婆甚至老公一口恶气吐到脸上来。如果不想费力气，那也不用做什么，大家场面上过的去就行了。
"""
naive_bayes.predict(TEST_TEXT)

training..
testing...
测试数据：420条
准确率：0.8714285714285714
星座
